In [2]:
import pandas as pd
import numpy as np
from pyvttbl import DataFrame
from pyvttbl.stats import Anova

##### SET STUDY VARIABLES
## Between-factors >> group on....
# Dep: Exptertise, FATR: Study
human = True
human = False

FATR = False
FATR = True


if FATR:
    print("** FATR Study **")
    source = os.path.join("data","fatr_students.csv")
    bfactor = "Study"
else:
    print("** Depend Study **")
    source = os.path.join("data","depend_only.csv")
    bfactor = "Expertise"


temp_csv_path = 'temp_cleaned_data.csv'

## Set DVs >> process multiple in one script, or just one at a time; 
dv_cols = ['Diagnosis Time']  
dv_cols = ['Cooper Harper']  
dv_cols = ['NasaTLX']
dv_cols = ['Decision']


# Create a pyvttbl.DataFrame from the cleaned pandas DataFrame
df = DataFrame()
df_pandas = pd.read_csv(source)

# Calculate NasaTLX if this is the DV for analysis
if dv_cols[0] != "NasaTLX":
    ## Drop rows with missing values in relevant columns
    # CAUTION! If dv_cols list contains multiple columns, then columns missing row values will obliterate other columns' row values
    df_pandas = df_pandas.dropna(subset=dv_cols)
    temp_csv_path = 'temp_cleaned_data.csv'
    df_pandas.to_csv(temp_csv_path, index=False)
else:
    print("create NasaTLX composit measure...")
    # Invert 'Performance' scores
    df_pandas['Performance'] = 11 - df_pandas['Performance']
    columns = ['Performance', 'Temporal Demand', 'Frustration Level', 'Mental Demand', 'Effort']
    for column in columns:
        median = df_pandas[column].median()
        df_pandas[column].fillna(median)
    # Calculate the NasaTLX composite score
    df_pandas['NasaTLX'] = df_pandas[columns].mean(axis=1)
    df_pandas.to_csv(temp_csv_path, index=False)

df.read_tbl(temp_csv_path)


    
for dv in dv_cols:
    print(f"Running ANOVA for: {dv}")
    
    aov = Anova()
    aov.run(df, dv, wfactors=['Scenario', 'Fault'], bfactors=[bfactor])   
    aov.truncate(test='gg')

    if human:
        ### HUMAN READABLE
        print(aov)
        print("\n\n") # this puts spaces between reports when running multiples
    else:
        ### For the Robots
        print(dict(aov))
        print("\n\n") 
    
        marginal_means = aov.get_marginal_means(['Scenario', 'Fault', bfactor])
        print(f"Marginal Means for: {dv}")
        print(marginal_means)
        print("\n\n") 



yes fatr
Running ANOVA for: Decision
('Scenario',)
('Study',)
('Fault', 'Study')
('Scenario', 'Fault', 'Study')
{('Fault',): {'y2': array([0.66666667, 0.83333333]), 'df': 1.0, 'eps_gg': 1.0, 'eps_hf': 1.0, 'eps_lb': 1.0, 'ss': 2.666666666666666, 'mss': 2.666666666666666, 'dfe': 22.0, 'sse': 4.791666666666667, 'mse': 0.2178030303030303, 'F': 12.243478260869562, 'p': 0.0020286288906186467, 'eta': 0.17391304347826084, 'obs': 48.0, 'critT': 2.073873067904015, 'se': 0.07127503522131787, 'ci': 0.139699069033783, 'lambda': 26.713043478260865, 'power': 0.9985217320274492, 'df_gg': 1.0, 'dfe_gg': 22.0, 'mss_gg': 2.666666666666666, 'mse_gg': 0.2178030303030303, 'F_gg': 12.243478260869562, 'p_gg': 0.0020286288906186467, 'obs_gg': 48.0, 'critT_gg': 2.073873067904015, 'se_gg': 0.07127503522131787, 'ci_gg': 0.139699069033783, 'lambda_gg': 26.713043478260865, 'power_gg': 0.9985217320274492, 'df_hf': 1.0, 'dfe_hf': 22.0, 'mss_hf': 2.666666666666666, 'mse_hf': 0.2178030303030303, 'F_hf': 12.24347826086

In [34]:
import os
import pandas as pd
from pyvttbl import DataFrame
from pyvttbl.stats import Anova
import numpy as np
import pprint

# Define study variables
# Set to True for FATR study, False for Depend study
STUDY_FATR = False  
STUDY_FATR = True
# True for human-readable output, False for structured data
HUMAN_READABLE = True  
HUMAN_READABLE = False

## Set DVs >> process multiple in one list, or just one at a time seems easier... 
DV = 'Diagnosis Time'
DV = 'Cooper Harper'
DV = 'NasaTLX'
DV = 'Decision'

# Paths and filenames
DATA_DIR = 'data'
TEMP_CSV_PATH = 'temp_cleaned_data.csv'

# Determine study based on STUDY_FATR flag
if STUDY_FATR:
    print("** FATR Study **")
    source_filename = 'fatr_students.csv'
    between_factor = "Study"
else:
    print("** Depend Study **")
    source_filename = 'depend_only.csv'
    between_factor = "Expertise"

source_path = os.path.join(DATA_DIR, source_filename)

# Load and preprocess data
df_pandas = pd.read_csv(source_path)

if DV == "NasaTLX":
    print("Creating NasaTLX composite measure...")
    # Invert 'Performance' scores and fill missing values with median
    df_pandas['Performance'] = 11 - df_pandas['Performance']
    columns = ['Performance', 'Temporal Demand', 'Frustration Level', 'Mental Demand', 'Effort']
    for column in columns:
        median = df_pandas[column].median()
        df_pandas[column].fillna(median, inplace=True)
    # Calculate the NasaTLX composite score
    df_pandas['NasaTLX'] = df_pandas[columns].mean(axis=1)
else:
    # Drop rows with missing values in the DV
    df_pandas.dropna(subset=[DV], inplace=True)

# Save cleaned data to temporary CSV
df_pandas.to_csv(TEMP_CSV_PATH, index=False)

# Load data into pyvttbl DataFrame for ANOVA
df_pyvttbl = DataFrame()
df_pyvttbl.read_tbl(TEMP_CSV_PATH)

# ANOVA settings
wfactors = ['Scenario', 'Fault']
bfactors = [between_factor]

# Run ANOVA
print(f"Running ANOVA for: {DV}")
aov = Anova()
aov.run(df_pyvttbl, DV, wfactors=wfactors, bfactors=bfactors)
aov.truncate(test='gg')

# Human-readable output
if HUMAN_READABLE:
    print(aov)
    print("\n\n")


# Convert aov_dict and marginal_means to pretty-printed string format
aov_dict_str = pprint.pformat(aov_dict)
marginal_means_str = pprint.pformat(marginal_means)

# Combine the two strings with a separator for clarity
combined_summary = f"ANOVA Results:\n{aov_dict_str}\n\nMarginal Means:\n{marginal_means_str}"


# These variables now hold the raw data in whatever format they originally had
# For example, aov_dict could be a dictionary with various statistics,
# and marginal_means could be a pandas DataFrame or a dictionary

if not HUMAN_READABLE:
    # Directly printing without conversion
    print("AOV Dictionary:", aov_dict)
    print("Marginal Means:", marginal_means)
    print("\n\n")


** FATR Study **
Running ANOVA for: Decision
('Scenario',)
('Study',)
('Fault', 'Study')
('Scenario', 'Fault', 'Study')
AOV Dictionary: {('Fault',): {'y2': array([0.66666667, 0.83333333]), 'df': 1.0, 'eps_gg': 1.0, 'eps_hf': 1.0, 'eps_lb': 1.0, 'ss': 2.666666666666666, 'mss': 2.666666666666666, 'dfe': 22.0, 'sse': 4.791666666666667, 'mse': 0.2178030303030303, 'F': 12.243478260869562, 'p': 0.0020286288906186467, 'eta': 0.17391304347826084, 'obs': 48.0, 'critT': 2.073873067904015, 'se': 0.07127503522131787, 'ci': 0.139699069033783, 'lambda': 26.713043478260865, 'power': 0.9985217320274492, 'df_gg': 1.0, 'dfe_gg': 22.0, 'mss_gg': 2.666666666666666, 'mse_gg': 0.2178030303030303, 'F_gg': 12.243478260869562, 'p_gg': 0.0020286288906186467, 'obs_gg': 48.0, 'critT_gg': 2.073873067904015, 'se_gg': 0.07127503522131787, 'ci_gg': 0.139699069033783, 'lambda_gg': 26.713043478260865, 'power_gg': 0.9985217320274492, 'df_hf': 1.0, 'dfe_hf': 22.0, 'mss_hf': 2.666666666666666, 'mse_hf': 0.2178030303030303

In [33]:
import anthropic
from IPython.display import display, Markdown
api_key = os.getenv('ANTHROPIC_API_KEY')
client = anthropic.Anthropic()

model_switch = 0

model_map = {
    0: "claude-instant-1.2",
    1: "claude-3-opus-20240229",
}

model = model_map.get(model_switch, "claude-instant-1.2")


message = client.messages.create(
    model=model,
    max_tokens=1000,
    temperature=0,
    system="You will be provided with two pieces of information:\n\n    A dictionary containing the results of an ANOVA, including F-values, p-values, effect sizes, and other relevant statistics for main effects, interactions, and other sources of variance.\n    A dictionary containing the estimated marginal means and standard errors for each factor and their combinations.\n\nYour task is to create a well-structured and formatted LaTeX report that interprets the ANOVA results and includes the estimated marginal means and standard errors. The report should follow this outline:\n\n    If the data contains results which are not significant, then disregard that variable. The statistical significance score for p=0.05 determines what needs to be reported.\n\n    Begin with a brief introductory statement about the effects on the dependent variable.\n    Report the main effects, including:\n        F-values, p-values, and effect sizes for each main effect\n        Interpretation of the estimated marginal means and standard errors for each level of the main effects\n    Report the two-way interactions, including:\n        F-values, p-values, and effect sizes for each two-way interaction\n        Interpretation of the estimated marginal means and standard errors for each combination of the interacting factors\n    Report the three-way interaction, if present, including:\n        F-value, p-value, and effect size for the three-way interaction\n        Interpretation of the estimated marginal means and standard errors for each combination of the interacting factors\n    Conclude with a summary statement highlighting the key findings and the complex interplay among the factors.\n\nUse the following LaTeX formatting guidelines:\n\n    Use the description environment for the overall structure\n    Use paragraph for the main sections (e.g., Main effects, Two-way interactions)\n    Use itemize for individual findings within each section\n    Use appropriate LaTeX commands for mathematical symbols and equations (e.g., $F(1, 22) = 15.30$, $p < .001$, $\\eta^2 = .13$)\n    Indent the LaTeX code for better readability and structure\n\nProvide the complete LaTeX code, without preamble and document environment. The output should be formatted in a raw markdown panel for easy copy.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": combined_summary
                }
            ]
        }
    ]
)




# HTML template for the code block
print(f'Results from Claude {model} for LaTeX')

html_template = f"""
<div style="background-color: #f6f8fa; padding: 10px; border-left: 6px solid #ccc; font-family: monospace; overflow: auto;">
<pre><code style="color: #333;">{message_formatted}</code></pre>
</div>
"""

display(HTML(html_template))



Results from Claude claude-instant-1.2 for LaTeX


In [29]:
# Example function to summarize ANOVA results and marginal means
def summarize_results(aov_dict, marginal_means):
    # Placeholder for the summary. You'd fill this in with actual data extraction and formatting
    summary = "ANOVA Summary:\n"
    for key, value in aov_dict.items():
        summary += f"{key}: {value}\n"
    
    summary += "\nMarginal Means:\n"
    for key, value in marginal_means.items():
        summary += f"{key}: {value}\n"
    
    # This is a simplified example. You'd replace it with actual summarization logic
    return summary

# Generate the summary text
summary_text = summarize_results(aov_dict, marginal_means)

# Proceed with your anthropic API call, inserting the summary_text into the 'text' field
import anthropic
from IPython.display import display, HTML

api_key = os.getenv('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(api_key=api_key)

model_switch = 0
model_map = {
    0: "claude-instant-1.2",
    1: "claude-3-opus-20240229",
}
model = model_map.get(model_switch, "claude-instant-1.2")

message = client.messages.create(
    model=model,
    max_tokens=1000,
    temperature=0,
    system="You will be provided with two pieces of information:\n\n    A dictionary containing the results of an ANOVA, including F-values, p-values, effect sizes, and other relevant statistics for main effects, interactions, and other sources of variance.\n    A dictionary containing the estimated marginal means and standard errors for each factor and their combinations.\n\nYour task is to create a well-structured and formatted LaTeX report that interprets the ANOVA results and includes the estimated marginal means and standard errors. The report should follow this outline:\n\n    If the data contains results which are not significant, then disregard that variable. The statistical significance score for p=0.05 determines what needs to be reported.\n\n    Begin with a brief introductory statement about the effects on the dependent variable.\n    Report the main effects, including:\n        F-values, p-values, and effect sizes for each main effect\n        Interpretation of the estimated marginal means and standard errors for each level of the main effects\n    Report the two-way interactions, including:\n        F-values, p-values, and effect sizes for each two-way interaction\n        Interpretation of the estimated marginal means and standard errors for each combination of the interacting factors\n    Report the three-way interaction, if present, including:\n        F-value, p-value, and effect size for the three-way interaction\n        Interpretation of the estimated marginal means and standard errors for each combination of the interacting factors\n    Conclude with a summary statement highlighting the key findings and the complex interplay among the factors.\n\nUse the following LaTeX formatting guidelines:\n\n    Use the description environment for the overall structure\n    Use paragraph for the main sections (e.g., Main effects, Two-way interactions)\n    Use itemize for individual findings within each section\n    Use appropriate LaTeX commands for mathematical symbols and equations (e.g., $F(1, 22) = 15.30$, $p < .001$, $\\eta^2 = .13$)\n    Indent the LaTeX code for better readability and structure\n\nProvide the complete LaTeX code, without preamble and document environment. The output should be formatted in a raw markdown panel for easy copy.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": summary_text  # Use the generated summary text here
                }
            ]
        }
    ]
)

# Assuming 'message_formatted' contains the response you want to display as code
message_formatted = "Your formatted message or response from Anthropic API here"

# HTML template for the code block
print(f'Results from Claude {model} for LaTeX')

html_template = f"""
<div style="background-color: #f6f8fa; padding: 10px; border-left: 6px solid #ccc; font-family: monospace; overflow: auto;">
<pre><code style="color: #333;">{message_formatted}</code></pre>
</div>
"""

display(HTML(html_template))


AttributeError: 'tuple' object has no attribute 'items'